In [1]:

import numpy as np
import pandas as pd


In [2]:
print('Loading dataset')
print('Checking is it in the data/external/folder')
import os
if [True for i in os.listdir('../../data/external/') if '.csv' in i] is None: 
    print('Running data aquisition script')
    %run data.external.RUNME
print('Ok.Loading dataset to notebook')

Loading dataset
Checking is it in the data/external/folder
Ok.Loading dataset to notebook


In [3]:
df = pd.read_csv('../../data/external/winequalityN.csv')


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [5]:
new_df = pd.DataFrame(data = scaler.fit_transform(df.iloc[:,1:-1]),columns=df.iloc[:,1:-1].columns)

In [6]:
new_df['quality'] = df.quality

In [7]:
new_df['type_white'] = df.type.apply(lambda x: 1 if x == 'white' else 0)

In [8]:
df = new_df

In [9]:
df_without_NA = pd.DataFrame()

# Fill missing 'fixed acidity' by mean same quality and wine type
df_without_NA['fixed acidity'] = df['fixed acidity'].fillna( 
                df.groupby(by=['quality','type_white'])['fixed acidity'].transform('mean') )

# Fill missing 'volatile acidity' by median same quality and wine type
df_without_NA['volatile acidity'] = df['volatile acidity'].fillna( 
                df.groupby(by=['quality','type_white'])['volatile acidity'].transform('median') )


# Fill missing 'citric acid' by median same quality and wine type
df_without_NA['citric acid'] = df['citric acid'].fillna( 
                df.groupby(by=['quality','type_white'])['citric acid'].transform('median') )

# Fill missing 'residual sugar' by mean same quality and wine type
df_without_NA['residual sugar'] = df['residual sugar'].fillna( 
                df.groupby(by=['quality','type_white'])['residual sugar'].transform('mean') )


# Fill missing 'chlorides' by mean same quality and wine type
df_without_NA['chlorides'] = df['chlorides'].fillna( 
                df.groupby(by=['quality','type_white'])['chlorides'].transform('mean') )


# Fill missing 'pH' by mean same quality and wine type
df_without_NA['pH'] = df['pH'].fillna( 
                df.groupby(by=['quality','type_white'])['pH'].transform('mean') )


# Fill missing 'sulphates' by mean same quality and wine type
df_without_NA['sulphates'] = df['sulphates'].fillna( 
                df.groupby(by=['quality','type_white'])['sulphates'].transform('mean') )

# make the other_acids
df_without_NA['other_acids'] = df_without_NA.apply(lambda x: x['fixed acidity'] - x['citric acid'],axis=1 )

In [10]:
#columns in original dataset (df) where is no missed data

features_wo_missed_values = [
    'free sulfur dioxide',
    'total sulfur dioxide',
    'density',
    'alcohol']


In [11]:
#preparing features
X = pd.concat( [df_without_NA,df[features_wo_missed_values] ],axis=1 )

#preparing target
a = {i:0 if i < 5 else 1 if i<9 else 2  for i in range(1,10) }
df['quality'] = df.quality.map(a)
df.quality.astype(int)
y = df['quality']


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,pH,sulphates,other_acids,free sulfur dioxide,total sulfur dioxide,density,alcohol
0,-0.167030,-0.423303,0.284180,3.206483,-0.315173,-1.358722,-0.545791,-0.451209,0.815565,0.959976,2.102214,-1.418558
1,-0.706883,-0.241083,0.146489,-0.808012,-0.200996,0.507693,-0.276979,-0.853372,-0.931107,0.287618,-0.232332,-0.831615
2,0.681310,-0.362563,0.559560,0.305958,-0.172452,0.258838,-0.612994,0.121750,-0.029599,-0.331660,0.134525,-0.328521
3,-0.012786,-0.666262,0.008799,0.642251,0.055902,-0.176659,-0.881807,-0.021585,0.928254,1.243074,0.301278,-0.496219
4,-0.012786,-0.666262,0.008799,0.642251,0.055902,-0.176659,-0.881807,-0.021585,0.928254,1.243074,0.301278,-0.496219


In [14]:
X.to_csv('X.csv')

In [ ]:
y.to_csv('y.csv')